In [1]:
import openmm as mm
from openmm import unit
from openmm import app
import numpy as np
from tqdm import tqdm

In [2]:
pdb = app.PDBFile('barnase_barstar.pdb')

In [3]:
type(pdb)

openmm.app.pdbfile.PDBFile

In [4]:
dir(pdb)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_atomNameReplacements',
 '_loadNameReplacementTables',
 '_numpyPositions',
 '_parseResidueAtoms',
 '_positions',
 '_residueNameReplacements',
 '_standardResidues',
 'getNumFrames',
 'getPositions',
 'getTopology',
 'positions',
 'topology',
 'writeFile',
 'writeFooter',
 'writeHeader',
 'writeModel']

In [5]:
positions = pdb.getPositions(asNumpy=True)

In [6]:
positions._value.shape

(3159, 3)

In [7]:
list_chains = []
for chain in pdb.topology.chains():
    list_chains.append(chain)

In [8]:
chain_0 = list_chains[0]

In [9]:
list_atoms_in_chain_0 = []
for atom in chain_0.atoms():
    list_atoms_in_chain_0.append(atom)

In [10]:
len(list_atoms_in_chain_0)

1727

In [11]:
chain_1 = list_chains[1]
list_atoms_in_chain_1 = []
for atom in chain_1.atoms():
    list_atoms_in_chain_1.append(atom)
    
len(list_atoms_in_chain_1)

1432

In [12]:
original_positions = pdb.getPositions(asNumpy=True)

In [13]:
barn=original_positions[0:1726]

In [14]:
bas=original_positions[1727:]

In [15]:
centro_0 = np.mean(original_positions[0:1726], 0)
centro_1 = np.mean(original_positions[1727:], 0)

In [16]:
r=centro_1-centro_0

In [17]:
forcefield = app.ForceField('amber14-all.xml')

In [18]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [19]:
step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

In [20]:
platform_name = 'CPU'
platform    = mm.Platform.getPlatformByName(platform_name)

In [21]:
context = mm.Context(system, integrator, platform)

In [22]:
type(context)

openmm.openmm.Context

In [23]:
context.setPositions(pdb.positions)

In [24]:
state = context.getState(getEnergy=True)

In [25]:
state.getPotentialEnergy()


Quantity(value=15294.695710703127, unit=kilojoule/mole)

In [26]:
coordenadas = pdb.getPositions(asNumpy=True)

In [27]:
norma=np.linalg.norm(r)
vec_normalizado=(r)/norma
paso=vec_normalizado*.05

In [28]:
a=int(r[2]/paso[2])
pos=[]

In [29]:
a

47

In [30]:
barn

Quantity(value=array([[-0.4722,  1.6032, -0.5051],
       [-0.4682,  1.66  , -0.6107],
       [-0.3589,  1.5705, -0.4824],
       ...,
       [ 0.4141,  3.4026,  0.6075],
       [ 0.3763,  3.3302,  0.5218],
       [ 0.5124,  3.4458,  0.5541]]), unit=nanometer)

In [31]:
paso

Quantity(value=array([ 0.0019567 , -0.02331077,  0.04419026]), unit=nanometer)

In [32]:
k=barn[0:]-paso
k

Quantity(value=array([[-0.4741567 ,  1.62651077, -0.54929026],
       [-0.4701567 ,  1.68331077, -0.65489026],
       [-0.3608567 ,  1.59381077, -0.52659026],
       ...,
       [ 0.4121433 ,  3.42591077,  0.56330974],
       [ 0.3743433 ,  3.35351077,  0.47760974],
       [ 0.5104433 ,  3.46911077,  0.50990974]]), unit=nanometer)

In [33]:
for i in range(0,46):
    pos.append(barn[0:]-(i*paso))

In [34]:
pos

[Quantity(value=array([[-0.4722,  1.6032, -0.5051],
        [-0.4682,  1.66  , -0.6107],
        [-0.3589,  1.5705, -0.4824],
        ...,
        [ 0.4141,  3.4026,  0.6075],
        [ 0.3763,  3.3302,  0.5218],
        [ 0.5124,  3.4458,  0.5541]]), unit=nanometer),
 Quantity(value=array([[-0.4741567 ,  1.62651077, -0.54929026],
        [-0.4701567 ,  1.68331077, -0.65489026],
        [-0.3608567 ,  1.59381077, -0.52659026],
        ...,
        [ 0.4121433 ,  3.42591077,  0.56330974],
        [ 0.3743433 ,  3.35351077,  0.47760974],
        [ 0.5104433 ,  3.46911077,  0.50990974]]), unit=nanometer),
 Quantity(value=array([[-0.47611341,  1.64982154, -0.59348052],
        [-0.47211341,  1.70662154, -0.69908052],
        [-0.36281341,  1.61712154, -0.57078052],
        ...,
        [ 0.41018659,  3.44922154,  0.51911948],
        [ 0.37238659,  3.37682154,  0.43341948],
        [ 0.50848659,  3.49242154,  0.46571948]]), unit=nanometer),
 Quantity(value=array([[-0.47807011,  1.67313231,

In [35]:
context.setPositions(coordenadas) ## Nunca actualizaste el objeto coordenadas. Sigue siendo el mismo que antes...

In [36]:
# Mira:
coordenadas

Quantity(value=array([[-0.4722,  1.6032, -0.5051],
       [-0.4682,  1.66  , -0.6107],
       [-0.3589,  1.5705, -0.4824],
       ...,
       [-0.6949,  0.7494,  4.3906],
       [-0.7008,  0.9331,  4.6818],
       [-0.7069,  0.8351,  4.7805]]), unit=nanometer)

In [37]:
state = context.getState(getEnergy=True)
state.getPotentialEnergy() # Por eso esta energía es la misma que antes

Quantity(value=15294.695710703127, unit=kilojoule/mole)

# Propuesta de Diego para continuar:

In [38]:
pdb = app.PDBFile('barnase_barstar.pdb')
original_positions = pdb.getPositions(asNumpy=True)

Veo cuantos átomos hay en total, cuantos son de una molécula y cuantos de otra:

In [39]:
num_total_atomos = original_positions.shape[0]

In [40]:
list_chains = list(pdb.topology.chains())
print(f'Tenemos {len(list_chains)} cadenas de átomos en el pdb')

Tenemos 2 cadenas de átomos en el pdb


In [41]:
num_atomos_cadena_1 = len(list(list_chains[0].atoms()))
num_atomos_cadena_2 = len(list(list_chains[1].atoms()))

print(f'En la cadena 1 tenemos {num_atomos_cadena_1} átomos')
print(f'En la cadena 2 tenemos {num_atomos_cadena_2} átomos')

En la cadena 1 tenemos 1727 átomos
En la cadena 2 tenemos 1432 átomos


Guardo los índices de los átomos de cada molécula y sus coordenadas originales:

In [42]:
indices_1 = list(range(0,num_atomos_cadena_1)) # Indices of molecule 1
indices_2 = list(range(num_atomos_cadena_1,num_total_atomos)) # Indices of molecule 2

In [43]:
original_positions_1=original_positions[indices_1]
original_positions_2=original_positions[indices_2]

Calculo el vector entre los dos centros geométricos y la distancia entre estos:

In [44]:
centro_1 = np.mean(original_positions_1, 0)
centro_2 = np.mean(original_positions_2, 0)

In [45]:
r=centro_2-centro_1

In [46]:
original_dist_centros = np.linalg.norm(r)

In [47]:
original_dist_centros

2.3962948699429214

La función np.linalg.norm nos quitó las unidades:

In [48]:
original_dist_centros = original_dist_centros * r.unit

In [49]:
original_dist_centros

Quantity(value=2.3962948699429214, unit=nanometer)

Calculo un vector normalizado entre centros:

In [50]:
vec_normalizado=r/original_dist_centros

In [51]:
vec_normalizado # Claro... no tiene unidades porque hemos dividido: nm/nm

array([ 0.03898872, -0.46648492,  0.88366945])

Para calcular la energía potencial del complejo necesito un objeto de tipo "Context" de OpenMM. "Context" lleva dentro las funciones necesarias para darme la energía dado un campo de fuerzas (parámetros para calcular el hamiltoniano del sistema):

In [52]:
forcefield = app.ForceField('amber14-all.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

platform_name = 'CPU'
platform    = mm.Platform.getPlatformByName(platform_name)

context = mm.Context(system, integrator, platform)

Para probar que funciona... veamos cuanto es la energía potencial del sistema con las moléculas dispuestas en el espacio como se encuentran en el fichero pdb:

In [53]:
context.setPositions(original_positions)
state = context.getState(getEnergy=True)
state.getPotentialEnergy()

Quantity(value=15294.695710703127, unit=kilojoule/mole)

Ahora, calculemos la energía potencial cuando una molécula es desplazada de la otra a lo largo de la dirección dada por el vector entre centros. Guardemos, como resultado final, dos listas: distancia entre centros y energía potencial.

In [74]:
distancias_entre_centros = []
energias_potenciales = []

Y vamos a desplazar N veces la molécula 2 a incrementos de longitud delta_x:

In [75]:
N = 1000
delta_x = 0.025 * unit.nanometers

Veamos:

In [76]:
for ii in tqdm(range(N)):
    temporal_position_2 = original_positions_2 + ii*delta_x*vec_normalizado
    new_positions = original_positions
    new_positions[indices_2,:] = temporal_position_2[:,:] # Actualizo las coordenadas de la molécula 2
    context.setPositions(new_positions)
    state = context.getState(getEnergy=True)
    energia_potencial= state.getPotentialEnergy() # Calculo la energía potencial
    # Guardo los valores
    distancias_entre_centros.append(original_dist_centros+ii*delta_x)
    energias_potenciales.append(energia_potencial)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:05<00:00, 15.35it/s]


Ya tenemos lo que queríamos:

In [77]:
distancias_entre_centros

[Quantity(value=2.3962948699429214, unit=nanometer),
 Quantity(value=2.4212948699429213, unit=nanometer),
 Quantity(value=2.4462948699429212, unit=nanometer),
 Quantity(value=2.4712948699429216, unit=nanometer),
 Quantity(value=2.4962948699429215, unit=nanometer),
 Quantity(value=2.5212948699429214, unit=nanometer),
 Quantity(value=2.5462948699429213, unit=nanometer),
 Quantity(value=2.5712948699429212, unit=nanometer),
 Quantity(value=2.5962948699429216, unit=nanometer),
 Quantity(value=2.6212948699429215, unit=nanometer),
 Quantity(value=2.6462948699429214, unit=nanometer),
 Quantity(value=2.6712948699429213, unit=nanometer),
 Quantity(value=2.6962948699429212, unit=nanometer),
 Quantity(value=2.7212948699429216, unit=nanometer),
 Quantity(value=2.7462948699429215, unit=nanometer),
 Quantity(value=2.7712948699429214, unit=nanometer),
 Quantity(value=2.7962948699429213, unit=nanometer),
 Quantity(value=2.8212948699429212, unit=nanometer),
 Quantity(value=2.8462948699429216, unit=nanom

In [78]:
energias_potenciales

[Quantity(value=15294.695710703127, unit=kilojoule/mole),
 Quantity(value=15376.03995840413, unit=kilojoule/mole),
 Quantity(value=15529.695914250966, unit=kilojoule/mole),
 Quantity(value=15832.972321405068, unit=kilojoule/mole),
 Quantity(value=16728.52349040517, unit=kilojoule/mole),
 Quantity(value=20127.691092358218, unit=kilojoule/mole),
 Quantity(value=32985.51471682285, unit=kilojoule/mole),
 Quantity(value=71025.52478120924, unit=kilojoule/mole),
 Quantity(value=134222.36185149683, unit=kilojoule/mole),
 Quantity(value=164615.30056380288, unit=kilojoule/mole),
 Quantity(value=155425.2446604572, unit=kilojoule/mole),
 Quantity(value=168805.92164383185, unit=kilojoule/mole),
 Quantity(value=154048.483335916, unit=kilojoule/mole),
 Quantity(value=88761.9117563325, unit=kilojoule/mole),
 Quantity(value=40417.53346498495, unit=kilojoule/mole),
 Quantity(value=22552.61823524605, unit=kilojoule/mole),
 Quantity(value=17804.54161661159, unit=kilojoule/mole),
 Quantity(value=16700.7233

## Tarea siguiente

Podrías representar gráficamente, con la ayuda de la librería de Python Matplotlib, la energía potencial (en Y) frente a la distancia entre centros(en X)? Podrías por favor poner el eje Y del gráfico con el límite inferior 15000 y el superior en 20000?

Hay un pequeño problema. Matplotlib no acepta listas de cantidades (con unidades), pero si una cantidad cuyo valor es una lista. Con la librería PyUnitWizard (nuestra y en desarrollo) podemos solucionar eso: 

In [79]:
import pyunitwizard as puw

In [80]:
energias_potenciales = puw.concatenate(energias_potenciales)
distancias_entre_centros = puw.concatenate(distancias_entre_centros)

Ya puedes hacer la representación gráfica.